In [1]:
import cv2
import os
import json
import numpy as np
import mediapipe as mp
from keras.utils import to_categorical

2024-08-24 21:14:18.446245: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-24 21:14:18.448577: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 21:14:18.582741: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-24 21:14:19.053181: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-24 21:14:20.189308: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warnin

In [2]:
path = "./archive"

In [3]:
def extract_data (json_list):
    data = []
    path = "./archive/videos/"
    for j in json_list:
        for ins in j["instances"] :
            video_id = ins["video_id"]
            video_path = f'{path}{video_id}.mp4'
            if os.path.exists(video_path) :
                data.append({"video_id" : video_id, "label" : j["gloss"]})
        if len(data) == 200 :
            return data

In [4]:
def extract_label (json_list) :
    labels = []
    for j in json_list:
        labels.append(j["gloss"])
    return labels

In [5]:
with open('WLASL_v0.3.json', 'r') as data_file:
    json_data = data_file.read()

json_list = json.loads(json_data)
data = extract_data(json_list)
labels = extract_label(json_list)

In [12]:
label_dict = dict()
label_rev = dict()
for l in range(len(labels)) :
    label_dict[labels[l]] = l
    label_rev[l] = labels[l]

In [ ]:
def extract_landmarks(data,target_frames = 50):
    x_data = []
    y_data = []
    count = 0
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic(static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)

    for d in data:

        path = f'./videos/{d["video_id"]}.mp4'
        cap = cv2.VideoCapture(path)
        frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        skip_frames = max(int(frames_count/target_frames), 1)

        for i in range(target_frames):
          cap.set(cv2.CAP_PROP_POS_FRAMES, i * skip_frames)
          success, frame = cap.read()

          if not success:
              break
          rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
          results = holistic.process(rgb_frame)
          frame_landmarks = []

          if (results.left_hand_landmarks or results.right_hand_landmarks) and results.pose_landmarks:
              for landmark in results.pose_landmarks.landmark:
                frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
              if results.left_hand_landmarks:
                for landmark in results.left_hand_landmarks.landmark:
                  frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
              else :
                frame_landmarks.extend([0,0,0]*21)
              if results.right_hand_landmarks :
                for landmark in results.right_hand_landmarks.landmark:
                    frame_landmarks.extend([landmark.x, landmark.y, landmark.z])
              else :
                frame_landmarks.extend([0,0,0]* 21)

              if len(frame_landmarks) > 0:
                x_data.append(frame_landmarks)
                y_data.append(d["label"])

        cap.release()
        count += 1
        print(f"Processed {count}/{len(data)} videos.")
    holistic.close()
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    np.save("./data/x_data.npy", x_data)
    np.save("./data/y_data.npy", y_data)

In [ ]:
extract_landmarks(data)

In [2]:
y_data = np.load("./data/y_data.npy")
labels = y_data.to_list

In [22]:
for i in range(len(labels)):
    labels[i] = label_dict[labels[i]]

In [24]:
y_data = np.array(labels)
y_data = y_data.reshape(40458,1)
y_data = to_categorical(y_data)